<a href="https://colab.research.google.com/github/cliang-wwf/bbs-trends/blob/main/BBS_trends_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BBS Route Trend Analysis
#### Data exploration and baseline model

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor

import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
bbs_path = '/content/combined_2004_2019_Route_level_BBS_RangeWide_trends_and_intercepts.csv'
ghf_path = '/content/Global_Human_Footprint_All_BBS_routes.csv'
ghf_change_path = '/content/Change_in_Global_Human_Footprint_All_BBS_routes.csv'
chf_path = '/content/Human_Footprint_Canadian_BBS_routes.csv'

bbs_df = pd.read_csv(bbs_path) # BBS route abundance and trend
ghf_df = pd.read_csv(ghf_path) # global human footprint for 4 diff years
ghfc_df = pd.read_csv(ghf_change_path) # route-level change in global human footprint (2005-2013)
chf_df = pd.read_csv(chf_path) # Canadian human footprint on each BBS route

In [15]:
bbs_data = bbs_df.copy()
hf_data = chf_df.copy()

In [29]:
bbs_data

,Unnamed: 0,english_name,BBS_route,BBS_stratum,Trend,lci95_Trend,uci95_Trend,half_CI_width,Mean_abundance,lci95_Mean_abundance,uci95_Mean_abundance,slope,lci95_slope,uci95_slope,sd_slope,Latitude,Longitude
0,1,Dickcissel,17-126,US-CO-18,13.992570,6.842509,22.023974,7.590733,0.114715,0.035410,0.259651,0.130155,0.066186,0.199047,0.040197,37.625667,-104.310066
1,2,Dickcissel,17-13,US-CO-18,11.665734,4.172726,19.324764,7.576019,0.228237,0.061297,0.530473,0.109501,0.040880,0.176679,0.040973,39.856886,-103.649377
2,3,Dickcissel,17-14,US-CO-18,16.538317,10.172034,23.305342,6.566654,1.658886,0.721360,3.160980,0.152470,0.096873,0.209494,0.034062,39.830769,-102.632591
3,4,Dickcissel,17-20,US-CO-18,13.892603,6.692818,21.761271,7.534227,0.358087,0.121919,0.755527,0.129301,0.064784,0.196892,0.039595,38.641057,-103.227936
4,5,Dickcissel,17-205,US-CO-18,14.248166,6.146271,23.039862,8.446796,0.190523,0.056466,0.420849,0.132222,0.059648,0.207338,0.044260,40.856616,-104.542015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90505,90506,Field Sparrow,64-31,US-ND-17,-2.177720,-9.337087,5.258217,7.297652,2.169468,1.142535,3.648500,-0.023001,-0.098022,0.051246,0.044345,47.407643,-101.931963
90506,90507,Field Sparrow,64-32,US-ND-17,2.439769,-3.659735,8.875875,6.267805,4.190165,2.753832,6.023837,0.023402,-0.037284,0.085038,0.037478,47.234060,-103.321358
90507,90508,Field Sparrow,64-33,US-ND-17,0.640947,-1.705494,3.072096,2.388795,23.363691,12.083068,38.995178,0.006286,-0.017202,0.030259,0.014361,47.617128,-103.545098
90508,90509,Field Sparrow,64-35,US-ND-17,2.431978,-3.541146,8.503644,6.022395,2.316137,1.423542,3.452137,0.023393,-0.036054,0.081614,0.035676,46.730988,-103.986594


In [ ]:
bbs_data.hist(['Trend', 'Mean_abundance', 'slope'], bins = 30)

In [ ]:
bbs_data.groupby(['english_name']).size().reset_index().rename(columns={0:'count'})

In [38]:
# BBS route-level trends for all-species model
sp = bbs_data.groupby('BBS_route').mean()
sp

,Unnamed: 0,Trend,lci95_Trend,uci95_Trend,half_CI_width,Mean_abundance,lci95_Mean_abundance,uci95_Mean_abundance,slope,lci95_slope,uci95_slope,sd_slope,Latitude,Longitude
BBS_route,,,,,,,,,,,,,,
11-1,43365.611111,-1.536589,-8.476286,5.745553,7.110919,7.157272,2.696527,14.794975,-0.017071,-0.089585,0.055467,0.044215,48.785290,-123.599130
11-10,43719.368421,-1.118128,-7.458847,5.485622,6.472234,5.945228,2.576612,11.329476,-0.012544,-0.078191,0.052907,0.040038,49.294880,-121.688470
11-101,43051.000000,-0.820372,-7.870523,6.556123,7.213323,5.938495,2.132434,12.452194,-0.009446,-0.082474,0.063396,0.044242,48.757438,-123.598512
11-102,46093.700000,0.326277,-7.377658,8.388641,7.883150,11.529684,2.806160,28.983833,0.001990,-0.076747,0.080366,0.047907,48.582066,-124.394624
11-11,44377.368421,0.053282,-7.689849,8.217836,7.953842,7.779407,3.037035,15.609143,-0.000994,-0.080531,0.078632,0.048575,49.184020,-122.534780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93-66,57316.818182,-1.703524,-9.957548,7.055133,8.506341,16.619135,7.153676,31.473273,-0.018862,-0.105576,0.067998,0.052472,64.088520,-140.957890
93-7,47820.100000,-1.025841,-8.313121,6.647101,7.480111,10.604693,3.724265,22.224750,-0.011430,-0.086971,0.064236,0.045807,60.086610,-130.406080
93-78,53660.615385,-3.311118,-8.883448,2.468836,5.676142,11.015760,4.027334,22.966747,-0.035649,-0.094607,0.022941,0.035867,66.442920,-136.710560


In [30]:
hf_data

,BBS_route,Rtname,Route,State,Name,bufferids,cumulative_400m_mean,cumulative_1km_mean,cumulative_2km_mean,cumulative_3km_mean,cumulative_4km_mean,cumulative_5km_mean,built_400m_mean,built_1km_mean,built_2km_mean,built_3km_mean,built_4km_mean,built_5km_mean,crop_400m_mean,crop_1km_mean,crop_2km_mean,crop_3km_mean,crop_4km_mean,crop_5km_mean,dam_and_associated_reservoir_400m_mean,dam_and_associated_reservoir_1km_mean,dam_and_associated_reservoir_2km_mean,dam_and_associated_reservoir_3km_mean,dam_and_associated_reservoir_4km_mean,dam_and_associated_reservoir_5km_mean,forestry_harvest_400m_mean,forestry_harvest_1km_mean,forestry_harvest_2km_mean,forestry_harvest_3km_mean,forestry_harvest_4km_mean,forestry_harvest_5km_mean,mines_400m_mean,mines_1km_mean,mines_2km_mean,mines_3km_mean,...,nav_water_2km_mean,nav_water_3km_mean,nav_water_4km_mean,nav_water_5km_mean,night_lights_400m_mean,night_lights_1km_mean,night_lights_2km_mean,night_lights_3km_mean,night_lights_4km_mean,night_lights_5km_mean,oil_gas_400m_mean,oil_gas_1km_mean,oil_gas_2km_mean,oil_gas_4km_mean,oil_gas_5km_mean,pasture_400m_mean,pasture_1km_mean,pasture_2km_mean,pasture_3km_mean,pasture_4km_mean,pasture_5km_mean,population_density_400m_mean,population_density_1km_mean,population_density_2km_mean,population_density_3km_mean,population_density_4km_mean,population_density_5km_mean,rail_400m_mean,rail_1km_mean,rail_2km_mean,rail_3km_mean,rail_4km_mean,rail_5km_mean,roads_400m_mean,roads_1km_mean,roads_2km_mean,roads_3km_mean,roads_4km_mean,roads_5km_mean,BBS_stratum
0,4-1,SEVEN PERSON,1,4,04-001 SEVEN PERSON,171,10.903027,9.600216,8.685464,8.173829,7.828887,7.543636,0.509383,0.283912,0.241026,0.202658,0.186441,0.171731,2.815013,2.811777,2.584615,2.532558,2.383051,2.269485,0.000000,0.000000,0.000000,0.00199,0.033882,0.063858,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.075067,0.079916,0.082564,0.072093,0.069734,0.073221,0.0,0.0,0.0,0.0,0.0,0.193029,0.151420,0.147692,0.126246,0.110412,0.096622,0.725130,0.727854,0.726531,0.753597,0.765722,0.754103,0.221024,0.219409,0.216632,0.212272,0.206196,0.216135,6.328841,5.371308,4.672485,4.273300,4.071636,3.896467,CA-AB-11
1,4-2,BOW ISLAND,2,4,04-002 BOW ISLAND,178,14.779228,13.744074,13.388046,13.096104,12.957665,12.755000,0.842391,0.560345,0.450689,0.339623,0.355089,0.320202,5.402174,5.604526,5.637858,5.837736,5.779945,5.743450,0.221024,0.227273,0.220339,0.19211,0.183862,0.174825,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.015504,0.047739,0.269022,0.269397,0.227996,0.187650,0.171543,0.166117,0.0,0.0,0.0,0.0,0.0,0.000000,0.008621,0.010604,0.013722,0.012003,0.010867,1.091897,0.988663,0.959020,0.926292,0.890465,0.877378,0.787062,0.582251,0.394068,0.319726,0.279790,0.255245,6.231806,5.500000,5.491525,5.279588,5.278041,5.162782,CA-AB-11
2,4-3,MILK RIVER,3,4,04-003 MILK RIVER,183,8.146986,7.066495,6.046228,5.498600,5.157840,4.749352,0.498615,0.224120,0.140041,0.116940,0.130036,0.117032,1.221607,1.322305,1.612033,1.695623,1.608108,1.445350,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.088643,0.074707,0.060166,0.060474,0.071137,0.059770,0.0,0.0,0.0,0.0,0.0,0.077562,0.068303,0.045643,0.037421,0.036716,0.030094,0.310474,0.309824,0.302957,0.295857,0.285201,0.273870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.940054,5.058698,3.885093,3.287387,3.026544,2.820201,CA-AB-11
3,4-4,FORT MACLEOD,4,4,04-004 FORT MACLEOD,187,10.672775,9.762944,8.779109,8.472579,8.606807,8.577107,0.530726,0.388027,0.328947,0.337238,0.365414,0.370813,1.720670,1.854767,1.577303,1.558040,1.718477,1.860247,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,

In [ ]:
hf_data.hist(bins=30, figsize=(25,20))
plt.show()

In [28]:
data = pd.merge(bbs_data, hf_data, on = 'BBS_route')

In [ ]:
data.dropna(inplace = True)
msno.matrix(data)

In [26]:
data

,Unnamed: 0,english_name,BBS_route,BBS_stratum_x,Trend,lci95_Trend,uci95_Trend,half_CI_width,Mean_abundance,lci95_Mean_abundance,uci95_Mean_abundance,slope,lci95_slope,uci95_slope,sd_slope,Latitude,Longitude,Rtname,Route,State,Name,bufferids,cumulative_400m_mean,cumulative_1km_mean,cumulative_2km_mean,cumulative_3km_mean,cumulative_4km_mean,cumulative_5km_mean,built_400m_mean,built_1km_mean,built_2km_mean,built_3km_mean,built_4km_mean,built_5km_mean,crop_400m_mean,crop_1km_mean,crop_2km_mean,crop_3km_mean,crop_4km_mean,crop_5km_mean,...,nav_water_2km_mean,nav_water_3km_mean,nav_water_4km_mean,nav_water_5km_mean,night_lights_400m_mean,night_lights_1km_mean,night_lights_2km_mean,night_lights_3km_mean,night_lights_4km_mean,night_lights_5km_mean,oil_gas_400m_mean,oil_gas_1km_mean,oil_gas_2km_mean,oil_gas_4km_mean,oil_gas_5km_mean,pasture_400m_mean,pasture_1km_mean,pasture_2km_mean,pasture_3km_mean,pasture_4km_mean,pasture_5km_mean,population_density_400m_mean,population_density_1km_mean,population_density_2km_mean,population_density_3km_mean,population_density_4km_mean,population_density_5km_mean,rail_400m_mean,rail_1km_mean,rail_2km_mean,rail_3km_mean,rail_4km_mean,rail_5km_mean,roads_400m_mean,roads_1km_mean,roads_2km_mean,roads_3km_mean,roads_4km_mean,roads_5km_mean,BBS_stratum_y
0,2232,Indigo Bunting,45-101,CA-MB-12,-2.797678,-7.530475,2.143360,4.836917,0.332637,0.145792,0.619438,-0.028858,-0.078291,0.021207,0.031051,49.44727,-95.40772,BRAINTREE,101,45,45-101 BRAINTREE,556,6.249334,4.771317,3.049927,2.246474,1.841221,1.616349,0.381471,0.164654,0.086440,0.056121,0.048432,0.039659,0.000000,0.000000,0.000000,0.000000,0.000000,0.012493,...,0.000000,0.000000,0.000000,0.000000,0.005450,0.002195,0.002161,0.004560,0.005608,0.005751,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.128786,0.116372,0.100738,0.087160,0.080407,0.076377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.452055,4.285714,2.685405,1.956583,1.591858,1.392581,CA-MB-12
1,7024,Rose-breasted Grosbeak,45-101,CA-MB-12,-4.536913,-8.239925,-0.654099,3.792913,4.548438,2.602640,7.146584,-0.046727,-0.085993,-0.006562,0.024347,49.44727,-95.40772,BRAINTREE,101,45,45-101 BRAINTREE,556,6.249334,4.771317,3.049927,2.246474,1.841221,1.616349,0.381471,0.164654,0.086440,0.056121,0.048432,0.039659,0.000000,0.000000,0.000000,0.000000,0.000000,0.012493,...,0.000000,0.000000,0.000000,0.000000,0.005450,0.002195,0.002161,0.004560,0.005608,0.005751,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.128786,0.116372,0.100738,0.087160,0.080407,0.076377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.452055,4.285714,2.685405,1.956583,1.591858,1.392581,CA-MB-12
2,11600,Scarlet Tanager,45-101,CA-MB-12,-3.172212,-8.119930,2.069189,5.094559,0.304747,0.113438,0.623234,-0.032746,-0.084686,0.020481,0.031944,49.44727,-95.40772,BRAINTREE,101,45,45-101 BRAINTREE,556,6.249334,4.771317,3.049927,2.246474,1.841221,1.616349,0.381471,0.164654,0.086440,0.056121,0.048432,0.039659,0.000000,0.000000,0.000000,0.000000,0.000000,0.012493,...,0.000000,0.000000,0.000000,0.000000,0.005450,0.002195,0.002161,0.004560,0.005608,0.005751,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.128786,0.116372,0.100738,0.087160,0.080407,0.076377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.452055,4.285714,2.685405,1.956583,1.591858,1.392581,CA-MB-12
3,14702,Canada Warbler,45-101,CA-MB-12,0.134793,-9.254575,9.852768,9.553672,0.342442,0.081057,0.876636,-0.000419,-0.097112,0.093971,0.059478,49.44727,-95.40772,BRAINTREE,101,45,45-101 BRAINTREE,556,6.249334,4.771317,3.049927,2.246474,1.841221,1.616349,0.381471,0.164654,0.086440,0.056121,0.048432,0.039659,0.000000,0.000000,0.000000,0.000000,0.000000,0.012493,...,0.000000,0.000000,0.000000,0.000000,0.005450,0.002195,0.002161,0.004560,0.005608,0.005751,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.128786,0.116372,0.100738,0.087160,0.080407,0.076377,0.000000,

In [32]:
x = data.iloc[:, 23:-1].values
y = data['Trend'].values #bbs_data['Mean_abundance']

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)